<a href="https://colab.research.google.com/github/S2H-Mobile/RoboND-DeepLearning-Project-Solution/blob/migrate-colab/code/model_training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

# Enable eager execution, for TensorFlow <2 only.
tf.enable_eager_execution()  

1.13.1


# Build and Compile the Model

In [0]:
def separable_conv2d_batchnorm(input_layer, filters, strides=1):
    output_layer = tf.keras.layers.SeparableConv2D(filters=filters,kernel_size=3, strides=strides,
                             padding='same', activation='relu')(input_layer)
    
    output_layer = tf.keras.layers.BatchNormalization()(output_layer) 
    return output_layer

def conv2d_batchnorm(input_layer, filters, kernel_size=3, strides=1):
    output_layer = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, 
                      padding='same', activation='relu')(input_layer)
    
    output_layer = tf.keras.layers.BatchNormalization()(output_layer) 
    return output_layer

In [0]:
def bilinear_upsample(input_layer, factor=2):
    return tf.keras.layers.UpSampling2D(size=(factor, factor), data_format=None, interpolation='bilinear')(input_layer)
    #return tf.keras.layers.BilinearUpSampling2D((factor, factor))(input_layer)

In [0]:
def encoder_block(input_layer, filters, strides):
    return separable_conv2d_batchnorm(input_layer, filters, strides)

In [0]:
def decoder_block(small_ip_layer, large_ip_layer, filters):
    
    # Upsample the small input layer using the bilinear_upsample() function.
    upsampled_layer = bilinear_upsample(small_ip_layer, factor=2)
    
    # Concatenate the upsampled and large input layers using layers.concatenate
    concatenated_layer = tf.keras.layers.concatenate([upsampled_layer, large_ip_layer])
    
    # Add two separable convolution layers
    conv_layer_1 = separable_conv2d_batchnorm(concatenated_layer, filters)
    conv_layer_2 = separable_conv2d_batchnorm(conv_layer_1, filters)
    return conv_layer_2

In [0]:
def fcn_model(inputs, num_classes):
    
    # Add Encoder Blocks. The filter size increases with each encoder layer.
    layer_1 = encoder_block(inputs, filters=32, strides=2)
    layer_2 = encoder_block(layer_1, filters=64, strides=2)

    # Add 1x1 Convolution layer using conv2d_batchnorm().
    layer_3 = conv2d_batchnorm(layer_2, filters=128, kernel_size=1, strides=1)
    
    # Add the same number of decoder blocks as the number of encoder blocks.
    layer_4 = decoder_block(layer_3, layer_1, filters=64)
    layer_5 = decoder_block(layer_4, inputs, filters=32)
    
    # Add output Layer. Convolution operates on the final layer in last decoder block.
    output = tf.keras.layers.Conv2D(num_classes, 1, activation='softmax', padding='same')(layer_5)
    
    # Return the output layer.
    return output

In [7]:
# Define the input layer
image_shape = (160, 160, 3)
input_layer = tf.keras.layers.Input(image_shape)

# Define the output layer
num_classes = 3
output_layer = fcn_model(inputs, num_classes)

NameError: ignored

In [0]:
# Define the model
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

# Print a summary
model.summary()

In [0]:
# Compile the model for training
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy')

# Data Preparation

In [0]:
# download data files from remote URL
!curl -o ./data/train.zip https://s3-us-west-1.amazonaws.com/udacity-robotics/Deep+Learning+Data/Lab/train.zip
!curl -o ./data/validation.zip https://s3-us-west-1.amazonaws.com/udacity-robotics/Deep+Learning+Data/Lab/validation.zip
!curl -o ./data/eval.zip https://s3-us-west-1.amazonaws.com/udacity-robotics/Deep+Learning+Data/Project/sample_evaluation_data.zip

In [0]:
!unzip -q ./data/train.zip -d ./data
!unzip -q ./data/validation.zip -d ./data
!unzip -q ./data/eval.zip -d ./data

# Training